# Notebook Setup

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
# Standard libraries
import numpy as np
import pandas as pd
import os
import io
import itertools

# Azure ML libraries
from azureml.core import Experiment, Workspace, Dataset

# Data preprocessing libraries
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff
import plotly.io as pio
from PIL import Image

%matplotlib inline
py.init_notebook_mode(connected=True)

In [3]:
## Orca configuration

# Open terminal and run "which orca" to find path of executable
# pio.orca.config.executable = "/anaconda/envs/customer-churn/bin/orca" # when running jupyter notebooks locally
pio.orca.config.executable = "/anaconda/envs/azureml_py36/bin/orca" # when running jupyter notebooks in Azure ML
# Configure plotly.py to run orca using Xvfb
pio.orca.config.use_xvfb = True
# Save config
pio.orca.config.save()

In [4]:
# Create directory to store images in run output
if not os.path.exists("outputs"):
    os.mkdir("outputs")

# Azure ML Setup

In [5]:
# Check core SDK version number
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK.\n")

# Load workspace into variable
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

# Create an Azure ML experiment in the workspace
experiment = Experiment(workspace = ws, name = "customer-churn-EDA-experiment")

You are currently using version 1.0.85 of the Azure ML SDK.

Workspace name: sbazuremlws
Azure region: westeurope
Subscription id: bf088f59-f015-4332-bd36-54b988be7c90
Resource group: sbazuremlrg


In [6]:
# Start logging data in the experiment
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

Starting experiment: customer-churn-EDA-experiment


# Data

In [7]:
# Load data from Azure ML dataset (authentication to underlying datastore via SAS token or access key)
dataset_name = 'customer-churn'

customerchurn = Dataset.get_by_name(workspace=ws, name=dataset_name)

# Load the Tabular Dataset into pandas DataFrame
telcom = customerchurn.to_pandas_dataframe()

In [8]:
# Display data
telcom.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,PartitionDate
PartitionDate,,,,,,,,,,,,,,,,,,,,,
2020-03-31,7590-VHVEG,Female,0,True,False,1,False,No phone service,DSL,No,...,No,No,No,Month-to-month,True,Electronic check,29.85,29.85,False,2020-03-31
2020-03-31,5575-GNVDE,Male,0,False,False,34,True,No,DSL,Yes,...,No,No,No,One year,False,Mailed check,56.95,1889.50,False,2020-03-31
2020-03-31,3668-QPYBK,Male,0,False,False,2,True,No,DSL,Yes,...,No,No,No,Month-to-month,True,Mailed check,53.85,108.15,True,2020-03-31
2020-03-31,7795-CFOCW,Male,0,False,False,45,False,No phone service,DSL,Yes,...,Yes,No,No,One year,False,Bank transfer (automatic),42.30,1840.75,False,2020-03-31
2020-03-31,9237-HQITU,Female,0,False,False,2,True,No,Fiber optic,No,...,No,No,No,Month-to-month,True,Electronic check,70.70,151.65,True,2020-03-31


In [9]:
# Show summary stats
telcom.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7044.000000,7044.000000,7044.000000,7033.00000
mean,0.162124,32.366695,64.755977,2282.98003
std,0.368590,24.560582,30.091735,2266.76945
min,0.000000,0.000000,18.250000,18.80000
25%,0.000000,9.000000,35.500000,401.30000
50%,0.000000,29.000000,70.350000,1397.30000
75%,0.000000,55.000000,89.850000,3794.50000
max,1.000000,72.000000,118.750000,8684.80000


In [10]:
# Show data types
telcom.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 7044 entries, 2020-03-31 to 2020-04-01
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        7044 non-null   object        
 1   gender            7044 non-null   object        
 2   SeniorCitizen     7044 non-null   int64         
 3   Partner           7044 non-null   bool          
 4   Dependents        7044 non-null   bool          
 5   tenure            7044 non-null   int64         
 6   PhoneService      7044 non-null   bool          
 7   MultipleLines     7044 non-null   object        
 8   InternetService   7044 non-null   object        
 9   OnlineSecurity    7044 non-null   object        
 10  OnlineBackup      7044 non-null   object        
 11  DeviceProtection  7044 non-null   object        
 12  TechSupport       7044 non-null   object        
 13  StreamingTV       7044 non-null   object        
 14  Stream

In [11]:
# Drop partition date colummn as it is redundant
telcom.drop(["PartitionDate"], axis=1, inplace=True)

In [12]:
# Data overview stats
row_count = telcom.shape[0]
column_count = telcom.shape[1]
columns = telcom.columns.tolist()
missing_values_total = telcom.isnull().sum().values.sum()

print ("Row Count     :" , row_count)
print ("Column Count  :" , column_count)
print ("\nColumns:")
print (columns)
print ("\nMissing Values Total:", missing_values_total) # sum over all columns

run.log('Row Count', row_count)
run.log('Column Count', column_count)
run.log_list('Columns', columns)
run.log('Missing Values Total', missing_values_total)

Row Count     : 7044
Column Count  : 21

Columns:
['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Missing Values Total: 11


In [13]:
# More data overview stats
missing_values_detail = telcom.isnull().sum()

for index in range(len(missing_values_detail)):
    print ("\nMissing values For Column", missing_values_detail.keys()[index], ":", missing_values_detail.values[index])
    run.log_row('Missing Values Per Column', x=missing_values_detail.keys()[index], y=missing_values_detail.values[index])
    
print("\n")

unique_values = telcom.nunique()

for index in range(len(unique_values)):
    print ("\nUnique Values For Column", unique_values.keys()[index], ":", unique_values.values[index])
    run.log_row('Unique Values Per Column', x=unique_values.keys()[index], y=unique_values.values[index])


Missing values For Column customerID : 0

Missing values For Column gender : 0

Missing values For Column SeniorCitizen : 0

Missing values For Column Partner : 0

Missing values For Column Dependents : 0

Missing values For Column tenure : 0

Missing values For Column PhoneService : 0

Missing values For Column MultipleLines : 0

Missing values For Column InternetService : 0

Missing values For Column OnlineSecurity : 0

Missing values For Column OnlineBackup : 0

Missing values For Column DeviceProtection : 0

Missing values For Column TechSupport : 0

Missing values For Column StreamingTV : 0

Missing values For Column StreamingMovies : 0

Missing values For Column Contract : 0

Missing values For Column PaperlessBilling : 0

Missing values For Column PaymentMethod : 0

Missing values For Column MonthlyCharges : 0

Missing values For Column TotalCharges : 11

Missing values For Column Churn : 0



Unique Values For Column customerID : 7044

Unique Values For Column gender : 2

Uniq

# Data Cleansing Part 1

In [14]:
# Replacing spaces with null values in total charges column
telcom['TotalCharges'] = telcom["TotalCharges"].replace(" ",np.nan)

# Dropping total charges column null values which contains .15% missing data (other option could for example be imputation)
telcom = telcom[telcom["TotalCharges"].notnull()]
telcom = telcom.reset_index()[telcom.columns]

# Convert total charges to float type
telcom["TotalCharges"] = telcom["TotalCharges"].astype(float)

# Replace 'No internet service' to No for the following columns
replace_cols = ["OnlineSecurity", "OnlineBackup", "DeviceProtection",
                "TechSupport","StreamingTV", "StreamingMovies"]
for i in replace_cols : 
    telcom[i]  = telcom[i].replace({"No internet service" : "No"})

# Replace values for Senior Citizen column
telcom["SeniorCitizen"] = telcom["SeniorCitizen"].replace({1:"Yes",0:"No"})

# Replace values for Churn column
telcom["Churn"] = telcom["Churn"].replace({1:"Yes",0:"No"})

# Transform Tenure to categorical column

def tenure_lab(telcom) :    
    if telcom["tenure"] <= 12 :
        return "Tenure_0-12"
    elif (telcom["tenure"] > 12) & (telcom["tenure"] <= 24 ):
        return "Tenure_12-24"
    elif (telcom["tenure"] > 24) & (telcom["tenure"] <= 48) :
        return "Tenure_24-48"
    elif (telcom["tenure"] > 48) & (telcom["tenure"] <= 60) :
        return "Tenure_48-60"
    elif telcom["tenure"] > 60 :
        return "Tenure_gt_60"
    
telcom["tenure_group"] = telcom.apply(lambda telcom:tenure_lab(telcom), axis = 1)

# Separate churn and non churn customers
churn     = telcom[telcom["Churn"] == "Yes"]
not_churn = telcom[telcom["Churn"] == "No"]

# Separate categorical and numerical columns
Id_col     = ["customerID"]
target_col = ["Churn"]
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist()
cat_cols   = [x for x in cat_cols if x not in target_col]
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

# Exploratory Data Analysis (Continuation)

In [15]:
# Labels
lab = telcom["Churn"].value_counts().keys().tolist()

# Values
val = telcom["Churn"].value_counts().values.tolist()

trace = go.Pie(labels = lab ,
               values = val ,
               marker = dict(colors =  [ 'royalblue' ,'lime'],
                             line = dict(color = "white",
                                         width =  1.3)
                            ),
               rotation = 90,
               hoverinfo = "label+value+text",
               hole = .5
              )

layout = go.Layout(dict(title = "Customer Churn Target Column Data Proportion",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                       )
                  )

data = [trace]
fig = go.Figure(data = data,layout = layout)

image_path = "outputs/churn_proportion.png"
fig.write_image(image_path)

# Upload the file explicitly into run artifacts 
run.upload_file(name = image_path, path_or_stream = image_path)

py.iplot(fig)

In [16]:
# Function for pie plot for customer churn types
def plot_pie(column) :
    
    trace1 = go.Pie(values  = churn[column].value_counts().values.tolist(),
                    labels  = churn[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    domain  = dict(x = [0,.48]),
                    name    = "Churn Customers",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    hole    = .6
                   )
    trace2 = go.Pie(values  = not_churn[column].value_counts().values.tolist(),
                    labels  = not_churn[column].value_counts().keys().tolist(),
                    hoverinfo = "label+percent+name",
                    marker  = dict(line = dict(width = 2,
                                               color = "rgb(243,243,243)")
                                  ),
                    domain  = dict(x = [.52,1]),
                    hole    = .6,
                    name    = "Non Churn Customers" 
                   )


    layout = go.Layout(dict(title = column + " Distribution in Customer Churn",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            annotations = [dict(text = "Churn Customers",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .15, y = .5),
                                           dict(text = "Non Churn Customers",
                                                font = dict(size = 13),
                                                showarrow = False,
                                                x = .88,y = .5
                                               )
                                          ]
                           )
                      )
    data = [trace1,trace2]
    fig  = go.Figure(data = data,layout = layout)
    
    image_path = "outputs/pie_plot_" + column + ".png"
    fig.write_image(image_path)
    
    # Upload the file explicitly into artifacts 
    run.upload_file(name = image_path, path_or_stream = image_path)
    
    py.iplot(fig)


# Function for histogram for customer churn types
def histogram(column) :
    trace1 = go.Histogram(x  = churn[column],
                          histnorm= "percent",
                          name = "Churn Customers",
                          marker = dict(line = dict(width = .5,
                                                    color = "black"
                                                    )
                                        ),
                         opacity = .9 
                         ) 
    
    trace2 = go.Histogram(x  = not_churn[column],
                          histnorm = "percent",
                          name = "Non Churn Customers",
                          marker = dict(line = dict(width = .5,
                                              color = "black"
                                             )
                                 ),
                          opacity = .9
                         )
    
    data = [trace1,trace2]
    layout = go.Layout(dict(title =column + " Distribution in Customer Churn ",
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = column,
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                             title = "percent",
                                             zerolinewidth=1,
                                             ticklen=5,
                                             gridwidth=2
                                            ),
                           )
                      )
    fig  = go.Figure(data=data,layout=layout)
    
    image_path = "outputs/histogram_" + column + ".png"
    fig.write_image(image_path)
    
    # Upload the file explicitly into artifacts 
    run.upload_file(name = image_path, path_or_stream = image_path)
    
    py.iplot(fig)
    
    
# Function for scatter plot matrix for numerical columns in data
def scatter_matrix(df)  :
    
    df  = df.sort_values(by = "Churn" ,ascending = True)
    classes = df["Churn"].unique().tolist()
    classes
    
    class_code  = {classes[k] : k for k in range(2)}
    class_code

    color_vals = [class_code[cl] for cl in df["Churn"]]
    color_vals

    pl_colorscale = "Portland"

    pl_colorscale

    text = [df.loc[k,"Churn"] for k in range(len(df))]
    text

    trace = go.Splom(dimensions = [dict(label  = "tenure",
                                       values = df["tenure"]),
                                  dict(label  = 'MonthlyCharges',
                                       values = df['MonthlyCharges']),
                                  dict(label  = 'TotalCharges',
                                       values = df['TotalCharges'])],
                     text = text,
                     marker = dict(color = color_vals,
                                   colorscale = pl_colorscale,
                                   size = 3,
                                   showscale = False,
                                   line = dict(width = .1,
                                               color='rgb(230,230,230)'
                                              )
                                  )
                    )
    axis = dict(showline  = True,
                zeroline  = False,
                gridcolor = "#fff",
                ticklen   = 4
               )
    
    layout = go.Layout(dict(title  = 
                            "Scatter Plot Matrix for Numerical Columns for Customer Churn",
                            autosize = False,
                            height = 800,
                            width  = 800,
                            dragmode = "select",
                            hovermode = "closest",
                            plot_bgcolor  = 'rgba(240,240,240, 0.95)',
                            xaxis1 = dict(axis),
                            yaxis1 = dict(axis),
                            xaxis2 = dict(axis),
                            yaxis2 = dict(axis),
                            xaxis3 = dict(axis),
                            yaxis3 = dict(axis),
                           )
                      )
    data   = [trace]
    fig = go.Figure(data = data,layout = layout )
    
    image_path = "outputs/scatter_matrix" + ".png"
    fig.write_image(image_path)
    
    # Upload the file explicitly into artifacts 
    run.upload_file(name = image_path, path_or_stream = image_path)

    py.iplot(fig)

# For all categorical columns plot a pie
for i in cat_cols :
    plot_pie(i)

# For all numerical columns plot a histogram    
for i in num_cols :
    histogram(i)

# Plot scatter plot matrix
scatter_matrix(telcom)

In [17]:
# Customer Churn in tenure groups
tg_ch  =  churn["tenure_group"].value_counts().reset_index()
tg_ch.columns  = ["tenure_group","count"]
tg_nch =  not_churn["tenure_group"].value_counts().reset_index()
tg_nch.columns = ["tenure_group","count"]

# Bar plot - Churn
trace1 = go.Bar(x = tg_ch["tenure_group"]  , y = tg_ch["count"],
                name = "Churn Customers",
                marker = dict(line = dict(width = .5,color = "black")),
                opacity = .9)

# Bar plot - Non Churn
trace2 = go.Bar(x = tg_nch["tenure_group"] , y = tg_nch["count"],
                name = "Non Churn Customers",
                marker = dict(line = dict(width = .5,color = "black")),
                opacity = .9)

layout = go.Layout(dict(title = "Customer attrition in tenure groups",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "tenure group",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "count",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                       )
                  )
data = [trace1,trace2]
fig  = go.Figure(data=data,layout=layout)

image_path = "outputs/bar_churn_tenure_groups.png"
fig.write_image(image_path)

# Upload the file explicitly into artifacts 
run.upload_file(name = image_path, path_or_stream = image_path)

py.iplot(fig)

In [18]:
# Monthly charges and total charges by tenure and churn group

# Scatter plot monthly charges & total charges by tenure group

def plot_tenure_scatter(tenure_group,color) :
    tracer = go.Scatter(x = telcom[telcom["tenure_group"] == tenure_group]["MonthlyCharges"],
                        y = telcom[telcom["tenure_group"] == tenure_group]["TotalCharges"],
                        mode = "markers",marker = dict(line = dict(color = "black",
                                                                   width = .2),
                                                       size = 4 , color = color,
                                                       symbol = "diamond-dot",
                                                      ),
                        name = tenure_group,
                        opacity = .9
                       )
    return tracer

# Scatter plot monthly charges & total charges by churn group

def plot_churncharges_scatter(churn,color) :
    tracer = go.Scatter(x = telcom[telcom["Churn"] == churn]["MonthlyCharges"],
                        y = telcom[telcom["Churn"] == churn]["TotalCharges"],
                        mode = "markers",marker = dict(line = dict(color = "black",
                                                                   width = .2),
                                                       size = 4 , color = color,
                                                       symbol = "diamond-dot",
                                                      ),
                        name = "Churn - " + churn,
                        opacity = .9
                       )
    return tracer

trace1 = plot_tenure_scatter("Tenure_0-12","#FF3300")
trace2 = plot_tenure_scatter("Tenure_12-24","#6666FF")
trace3 = plot_tenure_scatter("Tenure_24-48","#99FF00")
trace4 = plot_tenure_scatter("Tenure_48-60","#996600")
trace5 = plot_tenure_scatter("Tenure_gt_60","grey")
trace6 = plot_churncharges_scatter("Yes","red")
trace7 = plot_churncharges_scatter("No","blue")

data1   = [trace1,trace2,trace3,trace4,trace5] 
data2   = [trace7,trace6]

# Layout
def layout_title(title) :
    layout = go.Layout(dict(title = title,
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = "Monthly charges",
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                         title = "Total Charges",
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                            height = 600
                           )
                      )
    return layout

layout1  = layout_title("Monthly Charges & Total Charges by Tenure Group")
layout2  = layout_title("Monthly Charges & Total Charges by Churn Group")
fig1 = go.Figure(data = data1,layout = layout1)
fig2 = go.Figure(data = data2,layout = layout2)

image_path = "outputs/scatter_plot_tenure_group.png"
fig1.write_image(image_path)

# Upload the file explicitly into artifacts 
run.upload_file(name = image_path, path_or_stream = image_path)

py.iplot(fig1)

image_path = "outputs/scatter_plot_churn_group.png"
fig2.write_image(image_path)

# Upload the file explicitly into artifacts 
run.upload_file(name = image_path, path_or_stream = image_path)

py.iplot(fig2)

In [19]:
# Average charges by tenure groups
avg_tgc = telcom.groupby(["tenure_group","Churn"])[["MonthlyCharges",
                                                    "TotalCharges"]].mean().reset_index()

#function for tracing 
def mean_charges(column,aggregate) :
    tracer = go.Bar(x = avg_tgc[avg_tgc["Churn"] == aggregate]["tenure_group"],
                    y = avg_tgc[avg_tgc["Churn"] == aggregate][column],
                    name = aggregate,marker = dict(line = dict(width = 1)),
                    text = "Churn"
                   )
    return tracer

#function for layout
def layout_plot(title,xaxis_lab,yaxis_lab) :
    layout = go.Layout(dict(title = title,
                            plot_bgcolor  = "rgb(243,243,243)",
                            paper_bgcolor = "rgb(243,243,243)",
                            xaxis = dict(gridcolor = 'rgb(255, 255, 255)',title = xaxis_lab,
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                            yaxis = dict(gridcolor = 'rgb(255, 255, 255)',title = yaxis_lab,
                                         zerolinewidth=1,ticklen=5,gridwidth=2),
                           )
                      )
    return layout
    

#plot1 - mean monthly charges by tenure groups
trace1  = mean_charges("MonthlyCharges","Yes")
trace2  = mean_charges("MonthlyCharges","No")
layout1 = layout_plot("Average Monthly Charges by Tenure Groups",
                      "Tenure group","Monthly Charges")
data1   = [trace1,trace2]
fig1    = go.Figure(data=data1,layout=layout1)

#plot2 - mean total charges by tenure groups
trace3  = mean_charges("TotalCharges","Yes")
trace4  = mean_charges("TotalCharges","No")
layout2 = layout_plot("Average Total Charges by Tenure Groups",
                      "Tenure group","Total Charges")
data2   = [trace3,trace4]
fig2    = go.Figure(data=data2,layout=layout2)

image_path = "outputs/bar_monthy_charges_tenure_group.png"
fig1.write_image(image_path)

# Upload the file explicitly into artifacts 
run.upload_file(name = image_path, path_or_stream = image_path)

py.iplot(fig1)

image_path = "outputs/bar_total_charges_tenure_group.png"
fig2.write_image(image_path)

# Upload the file explicitly into artifacts 
run.upload_file(name = image_path, path_or_stream = image_path)

py.iplot(fig2)

In [ ]:
# Monthly Charges, Total Charges and Tenure
# Copy data
tel_df = telcom.copy()

# Drop tenure column
tel_df = tel_df.drop(columns = "tenure_group",axis = 1)

trace1 = go.Scatter3d(x = churn["MonthlyCharges"],
                      y = churn["TotalCharges"],
                      z = churn["tenure"],
                      mode = "markers",
                      name = "Churn customers",
                      text = "Id : " + churn["customerID"],
                      marker = dict(size = 1,color = "red")
                     )

trace2 = go.Scatter3d(x = not_churn["MonthlyCharges"],
                      y = not_churn["TotalCharges"],
                      z = not_churn["tenure"],
                      name = "Non churn customers",
                      text = "Id : " + not_churn["customerID"],
                      mode = "markers",
                      marker = dict(size = 1,color= "green")
                     )


layout = go.Layout(dict(title = "Monthly charges,total charges & tenure in customer attrition",
                        scene = dict(camera = dict(up=dict(x= 0 , y=0, z=0),
                                                   center=dict(x=0, y=0, z=0),
                                                   eye=dict(x=1.25, y=1.25, z=1.25)),
                                     xaxis  = dict(title = "monthly charges",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'),
                                     yaxis  = dict(title = "total charges",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'
                                                  ),
                                     zaxis  = dict(title = "tenure",
                                                   gridcolor='rgb(255, 255, 255)',
                                                   zerolinecolor='rgb(255, 255, 255)',
                                                   showbackground=True,
                                                   backgroundcolor='rgb(230, 230,230)'
                                                  )
                                    ),
                        height = 700,
                       )
                  )
                  
data = [trace1,trace2]
fig  = go.Figure(data = data,layout = layout)

image_path = "outputs/3D_monthly_charges_total_charges_tenure.png"
fig.write_image(image_path)

# Upload the file explicitly into artifacts 
run.upload_file(name = image_path, path_or_stream = image_path)

py.iplot(fig)

# Data Cleansing Part 2

In [ ]:
# Exclude ID column
Id_col     = ['customerID']

# Target column
target_col = ["Churn"]

# Categorical feature columns
cat_cols   = telcom.nunique()[telcom.nunique() < 6].keys().tolist() # get columns with less than 6 unique values
cat_cols   = [x for x in cat_cols if x not in target_col] # exclude target column (which is also categorical)

# Numerical feature columns
num_cols   = [x for x in telcom.columns if x not in cat_cols + target_col + Id_col]

# Binary columns with two values
bin_cols   = telcom.nunique()[telcom.nunique() == 2].keys().tolist()

# Columns with more than two values
multi_cols = [i for i in cat_cols if i not in bin_cols]

# Label encoding of binary columns
le = LabelEncoder()
for i in bin_cols :
    telcom[i] = le.fit_transform(telcom[i])
    
#Duplicating columns for multi value columns
telcom = pd.get_dummies(data = telcom,columns = multi_cols)

# Scaling of numerical columns
std = StandardScaler()
scaled = std.fit_transform(telcom[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

# Dropping original values and merging scaled values for numerical columns
df_telcom_og = telcom.copy()
telcom = telcom.drop(columns = num_cols,axis = 1)
telcom = telcom.merge(scaled,left_index=True,right_index=True,how = "left")

In [ ]:
telcom.head()

In [ ]:
# Variable summaries

df_telcom_og = telcom.copy()

summary = (df_telcom_og[[i for i in df_telcom_og.columns if i not in Id_col]].
           describe().transpose().reset_index())

summary = summary.rename(columns = {"index" : "feature"})
summary = np.around(summary,3)

val_lst = [summary['feature'], summary['count'],
           summary['mean'],summary['std'],
           summary['min'], summary['25%'],
           summary['50%'], summary['75%'], summary['max']]

trace  = go.Table(header = dict(values = summary.columns.tolist(),
                                line = dict(color = ['#506784']),
                                fill = dict(color = ['#119DFF']),
                               ),
                  cells  = dict(values = val_lst,
                                line = dict(color = ['#506784']),
                                fill = dict(color = ["lightgrey",'#F5F8FF'])
                               ),
                  columnwidth = [200,60,100,100,60,60,80,80,80])
layout = go.Layout(dict(title = "Variable Summary"))
figure = go.Figure(data=[trace],layout=layout)

# fig.write_image("outputs/summary_variables.png")
py.iplot(figure)

In [ ]:
# Correlation Matrix
# Calculate correlation
correlation = telcom.corr()
# # Determine tick labels
matrix_cols = correlation.columns.tolist()
# Convert to array
corr_array  = np.array(correlation)

# Plot
trace = go.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Viridis",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = go.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 720,
                        width   = 800,
                        margin  = dict(r = 0 ,l = 210,
                                       t = 25,b = 210,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 9)),
                        xaxis   = dict(tickfont = dict(size = 9))
                       )
                  )

data = [trace]
fig = go.Figure(data=data,layout=layout)

image_path = "outputs/correlation_matrix_variables.png"
fig.write_image(image_path)
# upload the file explicitly into artifacts 
run.upload_file(name = image_path, path_or_stream = image_path)
py.iplot(fig)

In [ ]:
# Visualising data with principal components
pca = PCA(n_components = 2)

X = telcom[[i for i in telcom.columns if i not in Id_col + target_col]]
Y = telcom[target_col + Id_col]

principal_components = pca.fit_transform(X)
pca_data = pd.DataFrame(principal_components,columns = ["PC1","PC2"])
pca_data = pca_data.merge(Y,left_index=True,right_index=True,how="left")
pca_data["Churn"] = pca_data["Churn"].replace({1:"Churn",0:"Not Churn"})

def pca_scatter(target,color) :
    tracer = go.Scatter(x = pca_data[pca_data["Churn"] == target]["PC1"] ,
                        y = pca_data[pca_data["Churn"] == target]["PC2"],
                        name = target,mode = "markers",
                        marker = dict(color = color,
                                      line = dict(width = .5),
                                      symbol =  "diamond-open"),
                        text = ("Customer Id : " + 
                                pca_data[pca_data["Churn"] == target]['customerID'])
                       )
    return tracer

layout = go.Layout(dict(title = "Visualising data with principal components",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 1",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "principal component 2",
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        height = 600
                       )
                  )
trace1 = pca_scatter("Churn",'red')
trace2 = pca_scatter("Not Churn",'royalblue')
data = [trace2,trace1]
fig = go.Figure(data=data,layout=layout)

image_path = "outputs/pca.png"
fig.write_image(image_path)
# upload the file explicitly into artifacts 
run.upload_file(name = image_path, path_or_stream = image_path)
py.iplot(fig)

In [ ]:
# Binary variable distribution for customer churn

# Separate binary columns
bi_cs = telcom.nunique()[telcom.nunique() == 2].keys()
dat_rad = telcom[bi_cs]

# Plot radar chart for churn and non churn customers (binary variables)
def plot_radar(df,aggregate,title) :
    data_frame = df[df["Churn"] == aggregate] 
    data_frame_x = data_frame[bi_cs].sum().reset_index()
    data_frame_x.columns  = ["feature","yes"]
    data_frame_x["no"]    = data_frame.shape[0]  - data_frame_x["yes"]
    data_frame_x  = data_frame_x[data_frame_x["feature"] != "Churn"]
    
    #count of 1's(yes)
    trace1 = go.Scatterpolar(r = data_frame_x["yes"].values.tolist(),
                             theta = data_frame_x["feature"].tolist(),
                             fill  = "toself",name = "count of 1's",
                             mode = "markers+lines",
                             marker = dict(size = 5)
                            )
    #count of 0's(No)
    trace2 = go.Scatterpolar(r = data_frame_x["no"].values.tolist(),
                             theta = data_frame_x["feature"].tolist(),
                             fill  = "toself",name = "count of 0's",
                             mode = "markers+lines",
                             marker = dict(size = 5)
                            ) 
    layout = go.Layout(dict(polar = dict(radialaxis = dict(visible = True,
                                                           side = "counterclockwise",
                                                           showline = True,
                                                           linewidth = 2,
                                                           tickwidth = 2,
                                                           gridcolor = "white",
                                                           gridwidth = 2),
                                         angularaxis = dict(tickfont = dict(size = 10),
                                                            layer = "below traces"
                                                           ),
                                         bgcolor  = "rgb(243,243,243)",
                                        ),
                            paper_bgcolor = "rgb(243,243,243)",
                            title = title,height = 700))
    
    data = [trace2,trace1]
    fig = go.Figure(data=data,layout=layout)
    
    image_path = "outputs/radar_" + title + ".png"
    fig.write_image(image_path)
    # upload the file explicitly into artifacts 
    run.upload_file(name = image_path, path_or_stream = image_path)
    py.iplot(fig)

#plot
plot_radar(dat_rad,1,"Churn -  Customers")
plot_radar(dat_rad,0,"Non Churn - Customers")

In [ ]:
run.complete()

In [ ]:
## Links for Orca
# https://plot.ly/python/static-image-export/
# https://github.com/plotly/orca